In [2]:
import pandas as pd
import altair as alt

In [3]:
df = pd.read_excel("gender_demographics.xlsx")
summer_df = df[df.term.str.contains('summer')]

df_2 =  pd.read_excel("ethnicity_demographics.xlsx")
summer_df_2 = df_2[df_2.term.str.contains('summer')]

In [4]:
click = alt.selection_multi(encodings=['x'])

domain = ["summer 2017", "summer 2018", "summer 2019", "summer 2021", "summer 2022"]
range_ = ["#e6ab02", "#d95f02", "#7570b3", "#e7298a", "#66a61e"]

base = alt.Chart(summer_df).mark_bar(size=18).encode(
    y = alt.Y("count()", scale=alt.Scale(domain=(0, 300)),
             axis=alt.Axis(title='Number of Students')),
    x = alt.X('term', axis=alt.Axis(title = "Term")),
    color = alt.condition(click, 'term', alt.value('lightgray'), scale=alt.Scale(domain=domain, range=range_))
).add_selection(
    click
).properties(
    height=300,
    width=150
)

gender = alt.Chart(summer_df).mark_bar(size=18).encode(
    y = alt.Y("count()", scale=alt.Scale(domain=(0, 300)),
             axis=alt.Axis(title='')),
    x = alt.X("gender", axis=alt.Axis(title="Gender")),
    color = alt.Color("term", scale=alt.Scale(domain=domain, range=range_))
).transform_filter(
    click
).properties(
    height=300,
    width=130
)

ethnicity = alt.Chart(summer_df_2).mark_bar(size=18).encode(
    y = alt.Y("count()", scale=alt.Scale(domain=(0, 300)),
             axis=alt.Axis(title='')),
    x = alt.X("ethnicity", axis = alt.Axis(title="Ethnicity")), 
    color = alt.Color("term", scale=alt.Scale(domain=domain, range=range_))
).transform_filter(
    click
).properties(
    height=300,
    width=190
)

combo = base|gender|ethnicity

combo.properties(
    title={"text": "Students Gender and Ethnicity by Term",
          "subtitle": "Click on the bars in the term graph to highlight those students in the gender and ethnicity graphs"}
   
)

/Users/mckennaquam/opt/anaconda3/lib/python3.8/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning)
/Users/mckennaquam/opt/anaconda3/lib/python3.8/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.HConcatChart(...)

In [6]:
df_3 = pd.read_csv("average_attendance.csv")
summer_df_3 = df_3[df_3.Term.str.contains('summer')]


line = alt.Chart(summer_df_3).mark_line().encode(
    y = alt.Y("Average Attendace (%)", scale=alt.Scale(domain=(0, 100))),
    x= "Term",
    color=alt.value("#7570b3")
).properties(
    width = 200
)

summer_2020_df = pd.DataFrame({
    'Term': ["summer 2019", "summer 2021"],
    'Average Attendace (%)':  [74, 79],
})

summer_2020_chart = alt.Chart(summer_2020_df).mark_line(strokeDash=[4,4]).encode(
    y = 'Average Attendace (%)',
    x = 'Term',
    color=alt.value("#7570b3")
)

point = alt.Chart(summer_df_3).mark_point().encode(
    y = alt.Y("Average Attendace (%)", scale=alt.Scale(domain=(0, 100))),
    x= "Term", 
    color=alt.value("#7570b3")
).properties(
    width = 200
)

(line + summer_2020_chart + point).properties(
    title="Average Attendace Over Terms"
)

alt.LayerChart(...)

In [5]:
df_sayot = pd.read_excel("SAYO_T_formatted.xlsx")
df_sayot['Post_Test'] = df_sayot['Post_Test'].round(decimals = 2)
df_sayot['Pre_Test'] = df_sayot['Pre_Test'].round(decimals = 2)

In [7]:
domain = ['Interacting with Adults', 'Communication', 'Critical Thinking', "English Language Skills", "Math Skills",
         "Interacting with Peers", "Perseverance", "Self Regulation"]
range_ = ['#7570b3', '#7570b3', '#7570b3', '#1b9e77', '#e7298a', '#7570b3', '#7570b3', '#7570b3']

selection = alt.selection(type="single", fields=['Category_Avg'], bind='legend')
#selection = alt.selection_interval()

scatter = alt.Chart(df_sayot).mark_point().encode(
    x= alt.X('Pre_Test', scale=alt.Scale(domain=(0, 5)), title = "Pre BtC Assesment"),
    y= alt.Y('Post_Test', scale=alt.Scale(domain=(0, 5)), title = "Post BtC Assesment"),
    color = alt.Color("Category_Avg", scale=alt.Scale(domain=domain, range=range_), legend=alt.Legend(title = "Skill Catagory")), 
    opacity = alt.condition(selection, alt.value(1), alt.value(0))
).properties(
    width=400,
    height=400,
    title={"text": "Average Scores of Students in BtC '22 from SAYO-T",
          "subtitle": "Anything above dashed line is improment, select the legend to see diffrent tests"}
).add_selection(selection)

line = pd.DataFrame({
    'pre_scores': [0, 5],
    'post_scores':  [0, 5],
})

line_plot = alt.Chart(line).mark_line(strokeDash=[2,2], color= "lightgray").encode(
    x= 'pre_scores',
    y= 'post_scores',
)

df_count = df_sayot.groupby(['Category_Avg','Pre_Test','Post_Test']).size().reset_index(name="Count")
df_count = df_count.sort_values(by=["Count"], ascending=False)

ranked_text = alt.Chart(df_count).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    selection
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<22
). properties(
    width = 45
)

before = ranked_text.encode(text='Pre_Test').properties(title='Pre')
after = ranked_text.encode(text='Post_Test').properties(title='Post')
count = ranked_text.encode(text='Count').properties(title='Count')
text = alt.hconcat(before, after, count) # Combine data tables

(line_plot+scatter)| text

alt.HConcatChart(...)

In [ ]:
df_count = df_sayot.groupby(['Category _Avg','Pre_Test','Post_Test']).size().reset_index(name="Count")
df_count